# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [79]:
import os
# import libraries
from sqlalchemy import create_engine
import pandas as pd
# nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# scikit-learn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
# pickle 
import pickle

In [80]:
# load data from database
path = os.path.abspath(os.getcwd())
#print(path)
#tmp_str = 'sqlite:///{}'.format(path + database_filepath[7:])
engine = create_engine('sqlite:///{}'.format(path+'/DisasterResponse.db'))
df = pd.read_sql('SELECT * FROM {}'.format('DisasterResponse'), engine)
X = df.message
Y = df.drop(columns=['id','message','original','genre'])
Y['child_alone'][0]=1
#Y = df[['medical_help','military']]
category_names = Y.columns

In [96]:
X.head(10)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
5               Information about the National Palace-
6                       Storm at sacred heart of jesus
7    Please, we need tents and water. We are in Sil...
8      I would like to receive the messages, thank you
9    I am in Croix-des-Bouquets. We have health iss...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [87]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok.lower().strip())
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [91]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                         ('tfidf', TfidfTransformer()),
                         ('clf',MultiOutputClassifier(LogisticRegression(random_state=42, max_iter = 500)))
                         ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                    class_weight=None,
                 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [93]:
# predict on test data
y_pred = pipeline.predict(X_test)
for idx, col in enumerate(category_names):
    print('For category {}:'.format(col))
    print(classification_report(y_test[col], y_pred[:,idx]))

For category related:
              precision    recall  f1-score   support

           0       0.70      0.47      0.56      1516
           1       0.85      0.94      0.89      4992
           2       0.00      0.00      0.00        46

    accuracy                           0.82      6554
   macro avg       0.52      0.47      0.48      6554
weighted avg       0.81      0.82      0.81      6554

For category request:
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5389
           1       0.83      0.57      0.67      1165

    accuracy                           0.90      6554
   macro avg       0.87      0.77      0.81      6554
weighted avg       0.90      0.90      0.89      6554

For category offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6522
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6554
   macro a

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6358
           1       0.53      0.04      0.08       196

    accuracy                           0.97      6554
   macro avg       0.75      0.52      0.53      6554
weighted avg       0.96      0.97      0.96      6554

For category death:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6224
           1       0.92      0.25      0.39       330

    accuracy                           0.96      6554
   macro avg       0.94      0.62      0.69      6554
weighted avg       0.96      0.96      0.95      6554

For category other_aid:
              precision    recall  f1-score   support

           0       0.88      0.99      0.93      5686
           1       0.63      0.12      0.20       868

    accuracy                           0.87      6554
   macro avg       0.75      0.55      0.56      6554
weighted avg       0.85      0

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
# specify parameters for grid search
#parameters = {'clf__estimator__max_features' : [None, 'sqrt', 'log2'],
#              'clf__estimator__min_samples_leaf': [1, 2, 5],
#            'clf__estimator__n_estimators': [100, 200, 500]
#             }
parameters = {'vect__ngram_range' : [(1,1), (1,2),(1,3)],
              'tfidf__use_idf': [True, False]
             }


# create grid search object
pipeline_cv = GridSearchCV(pipeline, parameters, n_jobs=-1, cv=3, verbose=1)
pipeline_cv.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 15.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s finished
[Parallel(n_jobs=1)]: Us

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        prep

In [17]:
print(pipeline_cv.best_params_)

{'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
# predict on test data
y_pred = pipeline_cv.predict(X_test)
for idx,col in enumerate(category_names):
    print('For category {}:'.format(col))
    print(classification_report(y_test[col], y_pred[:,idx]))

For category related:
              precision    recall  f1-score   support

           0       0.72      0.48      0.57      1524
           1       0.85      0.95      0.89      4978
           2       0.50      0.02      0.04        52

    accuracy                           0.83      6554
   macro avg       0.69      0.48      0.50      6554
weighted avg       0.82      0.83      0.81      6554

For category request:
              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5402
           1       0.83      0.55      0.66      1152

    accuracy                           0.90      6554
   macro avg       0.87      0.76      0.80      6554
weighted avg       0.90      0.90      0.89      6554

For category offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro a

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

For category hospitals:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       0.00      0.00      0.00        81

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

For category shops:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# tbd

### 9. Export your model as a pickle file

In [21]:
model_filepath = './models/classifier.pkl'
outfile = open(model_filepath,'wb')
pickle.dump(pipeline, outfile)
outfile.close()

In [22]:
infile = open(model_filepath,'rb')
clf2 = pickle.load(infile)
infile.close()
# predict on test data
y_pred = clf2.predict(X_test)
for idx,col in enumerate(y_test):
    print('For category {}:'.format(col))
    print(classification_report(y_test[col], y_pred[:,idx]))

/usr/local/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultiOutputClassifier from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or 

For category related:
              precision    recall  f1-score   support

           0       0.83      0.61      0.71      1524
           1       0.88      0.96      0.92      4978
           2       0.50      0.02      0.04        52

    accuracy                           0.87      6554
   macro avg       0.74      0.53      0.55      6554
weighted avg       0.87      0.87      0.86      6554

For category request:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5402
           1       0.86      0.59      0.70      1152

    accuracy                           0.91      6554
   macro avg       0.89      0.79      0.83      6554
weighted avg       0.91      0.91      0.91      6554

For category offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6523
           1       0.00      0.00      0.00        31

    accuracy                           1.00      6554
   macro a

/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

For category hospitals:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       1.00      0.01      0.02        81

    accuracy                           0.99      6554
   macro avg       0.99      0.51      0.51      6554
weighted avg       0.99      0.99      0.98      6554

For category shops:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0

In [33]:
clf2.predict([X_test[1]])[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [52]:
import numpy as np
list(np.array(Y.corr()))

[array([1.        , 0.2358971 , 0.03496727, 0.43732089, 0.15281872,
        0.1194073 , 0.08763785, 0.07033773, 0.09577082, 0.13572812,
        0.18421545, 0.16180388, 0.06514026, 0.07985849, 0.05576115,
        0.09663101, 0.11359665, 0.20230214, 0.13715323, 0.11394509,
        0.12036163, 0.0748426 , 0.04062195, 0.05432393, 0.03526371,
        0.05679303, 0.11143668, 0.32295904, 0.15562923, 0.16670326,
        0.05422682, 0.16715438, 0.07469887, 0.12239334, 0.25478838]),
 array([ 0.2358971 ,  1.        , -0.03050251,  0.44802442,  0.1286932 ,
         0.106415  ,  0.0714273 ,  0.05695927, -0.04426634,  0.23927435,
         0.35118272,  0.28060278,  0.08211652,  0.06279809,  0.02787422,
         0.02408843,  0.00886673,  0.2439334 ,  0.021803  ,  0.02572096,
         0.06300443,  0.01021775,  0.01941237,  0.01638912,  0.03232597,
         0.02279738, -0.00268652, -0.01454644, -0.00545233, -0.04566378,
        -0.00896983,  0.00697157, -0.00176454, -0.01537966,  0.64793722]),
 array([ 

In [78]:
list(pd.DataFrame(Y.sum().sort_values(), columns=['count']).index)


['offer',
 'shops',
 'tools',
 'fire',
 'hospitals',
 'missing_people',
 'aid_centers',
 'clothing',
 'security',
 'cold',
 'electricity',
 'money',
 'search_and_rescue',
 'military',
 'refugees',
 'other_infrastructure',
 'death',
 'transport',
 'medical_products',
 'buildings',
 'other_weather',
 'water',
 'infrastructure_related',
 'medical_help',
 'floods',
 'shelter',
 'storm',
 'earthquake',
 'food',
 'other_aid',
 'request',
 'direct_report',
 'weather_related',
 'aid_related',
 'related']

In [ ]:
Y.sum().sort_values

Object `pd.series.sort_values` not found.
